# Mesh generation for tutorial 05

In [ ]:
import dolfinx.io
import gmsh
import mpi4py

In [ ]:
import multiphenicsx.io
import multiphenicsx.mesh

### Geometrical parameters

In [ ]:
r = 3
lcar = 1. / 4.

### Create mesh with gmsh

In [ ]:
gmsh.initialize()
gmsh.model.add("mesh")
p0 = gmsh.model.geo.addPoint(0.0, 0.0, 0.0, lcar)
p1 = gmsh.model.geo.addPoint(0.0, +r, 0.0, lcar)
p2 = gmsh.model.geo.addPoint(0.0, -r, 0.0, lcar)
c0 = gmsh.model.geo.addCircleArc(p1, p0, p2)
c1 = gmsh.model.geo.addCircleArc(p2, p0, p1)
curve_loop = gmsh.model.geo.addCurveLoop([c0, c1])
circle = gmsh.model.geo.addPlaneSurface([curve_loop])
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(1, [c0, c1], 1)
gmsh.model.addPhysicalGroup(2, [circle], 0)
gmsh.model.mesh.generate(2)

### Convert to a dolfinx mesh

In [ ]:
mesh, subdomains, boundaries = multiphenicsx.mesh.gmsh_to_fenicsx(gmsh.model, gdim=2)
gmsh.finalize()

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

In [ ]:
multiphenicsx.io.plot_mesh_tags(boundaries)

### Save mesh and boundaries

In [ ]:
with dolfinx.io.XDMFFile(mpi4py.MPI.COMM_WORLD, "circle.xdmf", "w") as file:
    file.write_mesh(mesh)
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    file.write_meshtags(boundaries)